In [1]:
import tensorflow as tf
tf.__version__

'2.13.0'

In [2]:
import keras
keras.__version__

'2.13.1'

In [ ]:
import tensorflow as tf

# Use load_model from tf.keras.models
load_model = tf.keras.models.load_model

import h5py
import json
import numpy as np

import pipelinemodels

# Localizer model

In [4]:
modelfile = 'localizer_2019.model'
localizermodel = load_model(modelfile, compile=False)

In [5]:
# save the model in HDF5 format
# save in the current directory
localizermodel.save('localizer_2019_keras2.h5')
# Save model weights
localizermodel.save_weights('localizer_2019_weights.h5')

### save attributes (only needed for localizer model)

## this was previously used in 'pipeline' to load attributes
with h5py.File(modelfile, "r") as f:
    localizermodel.class_labels = [s.decode() for s in (f["labels"])]
    localizermodel.thresholds = dict(
        list(zip(localizermodel.class_labels, f["default_thresholds"]))
    )
## now save the attributes to a json file
attributes_output_file = 'localizer_2019_attributes.json'
attributes = {
    "class_labels": localizermodel.class_labels,
    "thresholds": localizermodel.thresholds
}
with open(attributes_output_file, 'w') as f:
    json.dump(attributes, f)

/Users/jacob/miniconda3/envs/keras2/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Create new model and verify that it is the same

In [6]:
localizermodel_fn = pipelinemodels.get_conv_model(initial_channels=16)
localizermodel_fn.load_weights('localizer_2019_weights.h5')

In [7]:
pipelinemodels.compare_models(localizermodel, localizermodel_fn)

Using default input shape: (1000, 1000, 1)
1/1 [==============================] - 0s 328ms/step
The models are identical in architecture, weights, and predictions.


True

### Save model data for comparison with Keras 3 model

In [8]:
input_shape = (1000,1000,1)  # this is the image resolution, so choose anything for testing
test_data = np.random.rand(10, *input_shape).astype(np.float32)
pipelinemodels.SaveModelData(localizermodel, test_data, 'localizer_keras2')

1/1 [==============================] - 0s 250ms/step
Model data saved to 'localizer_keras2_model_data.pkl'


### Print model summary

In [9]:
print("Previous model")
localizermodel.summary()

Previous model
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d_10 (Conv2D)          (None, None, None, 16)    160       
                                                                 
 spatial_dropout2d_6 (Spati  (None, None, None, 16)    0         
 alDropout2D)                                                    
                                                                 
 conv2d_11 (Conv2D)          (None, None, None, 32)    4640      
                                                                 
 batch_normalization_7 (Bat  (None, None, None, 32)    128       
 chNormalization)                                                
                                                                 
 spatial_dropout2d_7 (Spati  (None, None, No

In [10]:
print("Model defined from function")
localizermodel_fn.summary()

Model defined from function
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 16)    160       
                                                                 
 spatial_dropout2d (Spatial  (None, None, None, 16)    0         
 Dropout2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    4640      
                                                                 
 batch_normalization (Batch  (None, None, None, 32)    128       
 Normalization)                                                  
                                                                 
 spatial_dropout2d_1 (Spati  (Non

# Decoder model

In [11]:
decodermodel = load_model('decoder_2019.model', compile=False)

In [12]:
# save model
decodermodel.save('decoder_2019_keras2.h5')

In [13]:
# save weights
decodermodel.save_weights('decoder_2019_weights.h5')

### Create new model and verify that it is the same

In [14]:
decodermodel_fn = pipelinemodels.get_custom_resnet()
decodermodel_fn.load_weights('decoder_2019_weights.h5')

In [15]:
pipelinemodels.compare_models(decodermodel, decodermodel_fn)

1/1 [==============================] - 0s 296ms/step
The models are identical in architecture, weights, and predictions.


True

### Save model data for comparison with Keras 3 model

In [16]:
input_shape = decodermodel.input_shape[1:] 
test_data = np.random.rand(10, *input_shape).astype(np.float32)
pipelinemodels.SaveModelData(decodermodel, test_data, 'decoder_keras2')

1/1 [==============================] - 0s 23ms/step
Model data saved to 'decoder_keras2_model_data.pkl'


### Print model summary

In [17]:
print("Previous model")
decodermodel.summary()

Previous model
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 res2a_branch2a (Conv2D)     (None, 32, 32, 16)           32        ['input_2[0][0]']             
                                                                                                  
 bn2a_branch2a (BatchNormal  (None, 32, 32, 16)           64        ['res2a_branch2a[0][0]']      
 ization)                                                                                         
                                                                                                  
 activation_60 (Activation)  (None, 32, 32, 16)           0         ['bn2a_b

In [18]:
print("Model defined from function")
decodermodel_fn.summary()

Model defined from function
Model: "custom_resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 res2a_branch2a (Conv2D)     (None, 32, 32, 16)           32        ['input_2[0][0]']             
                                                                                                  
 bn2a_branch2a (BatchNormal  (None, 32, 32, 16)           64        ['res2a_branch2a[0][0]']      
 ization)                                                                                         
                                                                                                  
 activation_150 (Activation  (None, 32, 32, 16)         